In [1]:
import yfinance as yf

token = 'BTC-USD'

data = yf.download(token, interval='1d', period='max')
data = data.droplevel('Ticker', axis=1)
data.to_csv('../data/BTC-USD.csv')
data.head()

C:\Users\ivoto\AppData\Local\Temp\ipykernel_5048\2096007986.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(token, interval='1d', period='max')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2014-09-17,457.334015,468.174011,452.421997,465.864014,21056800
2014-09-18,424.440002,456.859985,413.104004,456.859985,34483200
2014-09-19,394.795990,427.834991,384.532013,424.102997,37919700
2014-09-20,408.903992,423.295990,389.882996,394.673004,36863600
2014-09-21,398.821014,412.425995,393.181000,408.084991,26580100


In [5]:
import pandas as pd
from ta.trend import EMAIndicator, MACD
from ta.momentum import RSIIndicator

# Carga tus datos en 'data' con columna 'Close'
data = pd.read_csv('../data/BTC-USD.csv')

# EMAs
data['ema20'] = EMAIndicator(data['Close'], window=20).ema_indicator()
data['ema50'] = EMAIndicator(data['Close'], window=50).ema_indicator()

# RSIs
data['rsi10']  = RSIIndicator(data['Close'], window=10).rsi()
data['rsi30']  = RSIIndicator(data['Close'], window=30).rsi()
data['rsi200'] = RSIIndicator(data['Close'], window=200).rsi()

# MACD
macd_ind = MACD(data['Close'], window_fast=12, window_slow=26, window_sign=9)
data['macd']   = macd_ind.macd()
data['signal'] = macd_ind.macd_signal()
data['hist']   = macd_ind.macd_diff()

# Señal de subida de precio
data['Price-Up'] = (data['Close'].shift(-1) > data['Close']).astype(int)

# Limpieza y exportación
data.dropna(inplace=True)
data.to_csv('../data/BTC-I.csv', index=False)
data
print(data.tail())


            Date          Close           High            Low           Open  \
3987  2025-08-17  117453.062500  118595.773438  117279.523438  117492.789062   
3988  2025-08-18  116252.312500  117614.171875  114723.679688  117453.906250   
3989  2025-08-19  112831.179688  116764.500000  112730.398438  116241.859375   
3990  2025-08-20  114274.742188  114625.796875  112387.960938  112828.023438   
3991  2025-08-21  112291.054688  114691.304688  112291.054688  114269.484375   

           Volume          ema20          ema50      rsi10      rsi30  \
3987  45852169525  117529.131310  115032.614311  49.456598  54.355996   
3988  72787808090  117407.529518  115080.445612  45.102138  52.687407   
3989  71657600353  116971.686677  114992.239105  35.271018  48.315885   
3990  67993811526  116714.834821  114964.101971  41.272651  50.122314   
3991  59282014208  116293.522427  114859.276588  36.154344  47.749865   

         rsi200        macd      signal        hist  Price-Up  
3987  54.737126 

# Stream_Crypto_Data 

In [7]:
import json 
import websocket
import pandas as pd
import numpy as np

coin = 'btcusdt@kline_1m'
socket = 'wss://stream.binance.com:9443/stream?streams=' + coin
print(socket)


ModuleNotFoundError: No module named 'websocket'

In [ ]:
def extract_ohlcv(source):
    event_time = pd.to_datetime(source['data']['E'], unit='ms')
    open = source['data']['k']['o']
    high = source['data']['k']['h']
    low = source['data']['k']['l']
    close = source['data']['k']['c']   
    volume = source['data']['k']['v']

    data = {'Open': open, 'High': high, 'Low': low, 'Close': close, 'Volume': volume}

    df = pd.DataFrame(data, index=[event_time])
    df.index.name = 'timestamp'
    df = df.astype(float)
    df = df.reset_index()
    print(df)
    return df

def on_message(ws, message):
    message = json.loads(message)
    extract_ohlcv(message)

In [ ]:
ws = websocket.WebSocketApp(socket, on_message=on_message)
ws.run_forever()

                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:18.014  84573.53  84593.7  84494.26  84494.26  10.41565
                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:20.015  84573.53  84593.7  84494.26  84494.27  10.43499
                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:22.029  84573.53  84593.7  84494.26  84494.27  10.61498


True